<a href="https://colab.research.google.com/github/AnasHamoutni/Cartoonize-Real-Life-Videos/blob/main/Cartoonize_Real_life_Videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cartoonize Real Life Videos**

Hi, my name is **ANAS HAMOUTNI** and I am a Data Scientist from **Morocco**.
This mini-Colab-Project is about Cartoonizing Real Life Videos using "White-box Cartoon Representations".

This Project's Github Link: https://github.com/AnasHamoutni/Cartoonize-Real-Life-Videos

Github Original Link: https://github.com/SystemErrorWang/White-box-Cartoonization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title ##**Install all the necessary libraries** { display-mode: "form" }

!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
!pip install torchvision==0.5
!pip install torch==1.4

from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
import ffmpeg
import os.path as osp
import logging
import shutil
import re
import gc
from PIL import Image
from tqdm import tqdm
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline
clear_output()

In [ ]:
#@title ##**Upload video** { display-mode: "form" }
#@markdown *Enter a link to the video below (for example, YouTube or Twitter), or leave the **source_url** field blank (in this case, you will be asked to upload the video from your computer).*
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Downloading videos lasting longer than one minute is not recommended. In addition, don't upload video that have a "space" or a "dot" in the title.*

#@markdown *If there is an error during execution, then run this block again*

In [ ]:
#@title ##**Split the video into frames** { display-mode: "form" }
%cd /content/

frames_of_video = int(cv2.VideoCapture("Chefchaouen-StrollingAroundMorocco'sBlueCity.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("Chefchaouen-StrollingAroundMorocco'sBlueCity.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture("Chefchaouen-StrollingAroundMorocco'sBlueCity.mp4")
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("/content/drive/MyDrive/test/test_images/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

/content


In [ ]:
#@title ##**Cartoonize the Frames** { display-mode: "form" }

%cd /content/drive/MyDrive/test
import os
import cv2
import numpy as np
import tensorflow as tf 
import network
import guided_filter
from tqdm import tqdm



def resize_crop(image):
    h, w, c = np.shape(image)
    if min(h, w) > 720:
        if h > w:
            h, w = int(720*h/w), 720
        else:
            h, w = 720, int(720*w/h)
    image = cv2.resize(image, (w, h),
                       interpolation=cv2.INTER_AREA)
    h, w = (h//8)*8, (w//8)*8
    image = image[:h, :w, :]
    return image
    

def cartoonize(load_folder, save_folder, model_path):
    input_photo = tf.placeholder(tf.float32, [1, None, None, 3])
    network_out = network.unet_generator(input_photo)
    final_out = guided_filter.guided_filter(input_photo, network_out, r=1, eps=5e-3)

    all_vars = tf.trainable_variables()
    gene_vars = [var for var in all_vars if 'generator' in var.name]
    saver = tf.train.Saver(var_list=gene_vars)
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(model_path))
    name_list = os.listdir(load_folder)
    for name in tqdm(name_list):
        try:
            load_path = os.path.join(load_folder, name)
            save_path = os.path.join(save_folder, name)
            image = cv2.imread(load_path)
            image = resize_crop(image)
            batch_image = image.astype(np.float32)/127.5 - 1
            batch_image = np.expand_dims(batch_image, axis=0)
            output = sess.run(final_out, feed_dict={input_photo: batch_image})
            output = (np.squeeze(output)+1)*127.5
            output = np.clip(output, 0, 255).astype(np.uint8)
            cv2.imwrite(save_path, output)
        except:
            print('cartoonize {} failed'.format(load_path))


    

if __name__ == '__main__':
    model_path = 'saved_models'
    load_folder = 'test_images'
    save_folder = 'cartoonized_images'
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    cartoonize(load_folder, save_folder, model_path)
    
clear_output()   

In [ ]:
#@title ##**Collecting the Cartoonized Video file** { display-mode: "form" }
import os
import imageio

%cd /content/drive/MyDrive/test/

frames = []
img = os.listdir('cartoonized_images/')
img.sort()
for i in img:
  frames.append(imageio.imread("cartoonized_images/"+i))
frames = np.array(frames)
imageio.mimsave("cartoonized_video.mp4", frames, fps=30)

!cp -r cartoonized_video.mp4 /content/cartoonized_video.mp4
clear_output()